In [1]:
import pandas as pd 

In [22]:
products = pd.read_csv("data/olist_products_dataset.csv")
main = pd.read_csv("data/df_main.csv")

In [24]:
main.columns

Index(['Unnamed: 0', 'order_id', 'product_id', 'seller_id', 'customer_id',
       'item_count', 'order_purchase_timestamp',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'shipping_limit_date', 'price', 'freight_value',
       'product_category_name', 'product_weight_g', 'customer_zip_code',
       'customer_city', 'customer_state', 'seller_zip_code', 'seller_city',
       'seller_state', 'product_volume_cm^3', 'order_delivery_delta_time',
       'delivery_delay_delta_days', 'order_delivery_time',
       'order_estimated_delivery_time', 'delivery_distance'],
      dtype='object')

In [16]:
products

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40,287,1,225,16,10,14
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44,276,1,1000,30,18,20
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46,250,1,154,18,9,15
3,cef67bcfe19066a932b7673e239eb23d,bebes,27,261,1,371,26,4,26
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37,402,4,625,20,17,13


In [21]:
product_id = "3aa071139cb16b67ca9e5dea641aaa2f"
product = products[products["product_id"] == product_id]
print(product)
product_price = product["product_weight_g"]
product_name = product["product_category_name"].values[0]
print(product_name)

                         product_id product_category_name  \
1  3aa071139cb16b67ca9e5dea641aaa2f                 artes   

   product_name_lenght  product_description_lenght  product_photos_qty  \
1                   44                         276                   1   

   product_weight_g  product_length_cm  product_height_cm  product_width_cm  
1              1000                 30                 18                20  
artes


In [11]:
products.columns

Index(['product_id;product_category_name;product_name_lenght;product_description_lenght;product_photos_qty;product_weight_g;product_length_cm;product_height_cm;product_width_cm'], dtype='object')

In [18]:
for index, row in products.iterrows():
    print(row["product_id"])

TypeError: tuple indices must be integers or slices, not str

In [26]:
import requests
import numpy as np

In [ ]:
KEY = "bf698b456ad4cebada870914b2021fab"
def construct_api_call(query):
    url = f"http://api.positionstack.com/v1/forward?access_key={KEY}&query={query}"
    r = requests.get(url)
    body = r.json()
    #body["data"][0]["latitude"]
    #body["data"][0]["longitude"]
    try: 
        return body["data"][0]["latitude"] 
    except:
        return np.nan
    
query = "11547, cubatao"
url = construct_api_call(query)

In [30]:
url = "http://api.positionstack.com/v1/forward?access_key=bf698b456ad4cebada870914b2021fab&query=11547"
r = requests.get(url)
print(r)
body = r.json()
print(body)